# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability for free (ReAct reasoning, memory, multi‑step planning).
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.

## Roadmap
1. Environment setup
2. Write simple tools and connect them to an LLM
3. Standardize tool calling by writing `to_schema`
4. Use LangChain to augment an LLM with your tools
5. Build a Perplexity‑style web‑search agent
6. (Optional) A minimal backend and frontend UI

# 1- Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent_latest

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=web_agent_latest --display-name "web_agent_latest"
```
Once this is done, you can select “web_agent” from the Kernel → Change Kernel menu in Jupyter or VS Code.


> Behind the scenes:
> * Conda reads `environment.yml`, resolves the pinned dependencies, creates an isolated environment named `web_agent`, and activates it.
> * `ollama pull` downloads the model so you can run it locally without API calls.


## 1.2 Ollama setup

In this project, we start with `gemma3-1B` because it is lightweight and runs on most machines. You can try other smaller or larger LLMs such as `mistral:7b`, `phi3:mini`, or `llama3.2:1b` to compare performance. Explore available models here: https://ollama.com/library

```bash
ollama pull gemma3:1b
```

`ollama pull` downloads the model so you can run it locally without API calls.


## 2- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.


We describe each tool’s interface in the model’s prompt, defining what it does and what arguments it expects. When the model decides that a tool is needed, it emits a structured output like: `TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Francisco"}}`. Your code will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3` model how to use it when required in four steps:
1. Implement the tool
2. Create the instructions for the LLM
3. Call the LLM with the prompt
4. Parse the LLM output and call the tool

In [1]:
from openai import OpenAI

client = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

In [2]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# Your goal: give the model a way to access weather information.
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Requirements:
#   • The function should be named `get_current_weather`
#   • It should take two arguments:
#         - city: str
#         - unit: str = "celsius"
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#


def get_current_weather(city: str, unit: str = "celsius") -> str:
    #complete_url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
    """Weather for city."""
    return f"It is 23°C and sunny in {city}."

def get_stock_price(ticker: str) -> str:
    #complete_url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
    """Price of a stock."""
    return f"Stock Price for {ticker} is $56."



In [3]:
# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------
# Goal:
#   Build the system and user prompts that instruct the model when and how
#   to use your tool (`get_current_weather`).
#
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describe the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

# Try experimenting with different system and user prompts
# ---------------------------------------------------------

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users city.",
                    },
                },
                "required": ["city", "unit"]
            }
        }
    }
]

messages = []
messages.append({"role": "system", "content": "get_current_weather is a function that returns the current weather for a given city. Use this function to answer user questions about the weather."})
messages.append({"role": "user", "content": "What's the weather in Dhaka today?"})


Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

Start the **Ollama** server in a terminal with `ollama serve`. This launches a local API endpoint that listens for LLM requests. Once the server is running, return to the notebook and in the next cell send a query to the model.


In [4]:
# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Use the Ollama client to create a chat completion. 
#       - You may find some examples here: https://platform.openai.com/docs/api-reference/chat/create
#       - If you are unsure, search the web for "client.chat.completions.create"
#   2. Print the raw response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------

response = client.chat.completions.create(
    model="mistral:7b",
    messages=messages,
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "get_current_weather"}},
)
print(response)



ChatCompletion(id='chatcmpl-766', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_yzwuuvxk', function=Function(arguments='{"city":"Dhaka, Bangladesh","unit":"celsius"}', name='get_current_weather'), type='function', index=0)]))], created=1761196095, model='mistral:7b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=74, prompt_tokens=155, total_tokens=229, completion_tokens_details=None, prompt_tokens_details=None))


In [5]:
# ---------------------------------------------------------
# Step 4: Parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import json

response_message = response.choices[0].message
messages.append(response_message)  # Add the model's response to the conversation

# 6. Check if the model wants to call a function
if response_message.tool_calls:
    print("[Model] Requested a function call:\n")
    
    # In this example, we'll just use the first tool call.
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)

    print(f"  Function: {function_name}")
    print(f"  Arguments: {function_args}\n")

    # 7. Execute your local Python function
    if function_name == "get_current_weather":
        # Call the actual Python function
        function_response = get_current_weather(city=function_args.get("city"), unit=function_args.get("unit"))

        # 8. Send the function's result back to the model
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
        
        print(f"[Python] Sending function result back to model:\n  {function_response}\n")

        # 9. Get the final, natural-language response
        final_response = client.chat.completions.create(
            model="mistral:7b",
            messages=messages, # Send the *entire* conversation history
            temperature=0.1
        )
        
        print(f"[Model] {final_response.choices[0].message.content}")
else:
    # The model didn't need a function
    print(f"[Model] {response_message.content}")


[Model] Requested a function call:

  Function: get_current_weather
  Arguments: {'city': 'Dhaka, Bangladesh', 'unit': 'celsius'}

[Python] Sending function result back to model:
  It is 23°C and sunny in Dhaka, Bangladesh.

[Model]  It is currently 23 degrees Celsius and sunny in Dhaka, Bangladesh. Enjoy your day!


# 3- Standadize tool calling

So far, we handled tool calling manually by writing one regex and one hard-coded function. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the `TOOL_SPEC` prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function’s signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [6]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Use `inspect.signature` to get function parameters.
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing:
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect


def to_schema(fn):
    """Inspects a function and generates an OpenAI-compatible tool schema."""
    
    # A simple mapping from Python types to JSON Schema types
    type_mapping = {
        str: "string",
        int: "number",
        float: "number",
        bool: "boolean",
    }

    # Get the function's signature
    sig = inspect.signature(fn)
    
    # Get the function's docstring (description)
    # inspect.getdoc() cleans up indentation
    description = inspect.getdoc(fn)

    # Build the 'properties' and 'required' list for the JSON schema
    properties = {}
    required = []

    for name, param in sig.parameters.items():
        # Get the Python type hint
        param_type = param.annotation
        
        # Map Python type to JSON Schema type
        json_type = type_mapping.get(param_type, "string") # Default to string if type is unknown

        properties[name] = {
            "type": json_type,
            # Note: This simple version doesn't parse docstrings for parameter descriptions
            "description": f"The {name} parameter" 
        }

        # If the parameter has no default value, it's required
        if param.default == inspect.Parameter.empty:
            required.append(name)

    # Assemble the final schema
    schema = {
        "type": "function",
        "function": {
            "name": fn.__name__,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": properties,
                "required": required,
            },
        },
    }
    
    return schema

tool_schema = to_schema(get_current_weather)
tools = []
tools.append(tool_schema)
tools.append(to_schema(get_stock_price))
pprint(tools)

[{'function': {'description': 'Weather for city.',
               'name': 'get_current_weather',
               'parameters': {'properties': {'city': {'description': 'The city '
                                                                     'parameter',
                                                      'type': 'string'},
                                             'unit': {'description': 'The unit '
                                                                     'parameter',
                                                      'type': 'string'}},
                              'required': ['city'],
                              'type': 'object'}},
  'type': 'function'},
 {'function': {'description': 'Price of a stock.',
               'name': 'get_stock_price',
               'parameters': {'properties': {'ticker': {'description': 'The '
                                                                       'ticker '
                                                     

In [7]:
# ---------------------------------------------------------
# Provide the tool schema to the model
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the raw model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

messages = []
#messages.append({"role": "system", "content": "get_current_weather is a function that returns the current weather for a given city. Use this function to answer user questions about the weather."})
#messages.append({"role": "user", "content": "What's the weather in Dhaka today?"})

messages.append({"role": "system", "content": "Use the correct function to answer user questions."})
messages.append({"role": "user", "content": "What is the stock price of GM?"})
messages.append({"role": "user", "content": "What's the weather in Dhaka today?"})

response = client.chat.completions.create(
    model="mistral:7b",
    messages=messages,
    tools=tools,
    tool_choice="required",
)
print(response)

ChatCompletion(id='chatcmpl-971', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' [{"name":"get_stock_price","arguments": {"ticker": "GM"}}], [{"name":"get_current_weather","arguments": {"city": "Dhaka", "unit": "C"}}]\n\nIn terms of stock, the current price for General Motors (GM) is not specified here as I do not have real-time stock data. To check the current price, you may need to integrate a financial API or visit a financial website like Yahoo Finance.\n\nFor weather in Dhaka, today it\'s 36.4°C according to the information provided with this example. Real-world weather changes frequently so you should verify the temperature and time for accurate weather information by using Weather APIs such as OpenWeatherMap or Weatherbit.\n\nI encourage you to use the provided functions within a programming environment to get real-time results when available.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None

In [8]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Goal:
#   Convert your weather function into a LangChain-compatible tool.
#
# Steps:
#   1. Import `tool` from `langchain.tools`.
#   2. Keep your existing `get_current_weather` helper as before.
#   3. Create a new function (e.g., get_weather) that calls it.
#   4. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.
#   • Start with keeping the logic simple and offline-friendly.

from langchain_core.tools import tool


@tool
def get_weather(city: str, unit: str = "celsius") -> str:
    """Weather for a city."""
    return f"It is 23°C and sunny in {city}."

@tool
def get_stock_price(ticker: str) -> str:
    #complete_url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
    """Price of a stock."""
    return f"Stock Price for {ticker} is $56."

tools = []
tools.append(get_weather)
tools.append(get_stock_price)


In [9]:
# ---------------------------------------------------------
# Provide the tool schema to the model
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the raw model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

llm = ChatOllama(model="mistral:7b", temperature=0)
model_with_tools = llm.bind_tools(tools, tool_choice="get_weather")

while True:
    user_prompt = input("Enter a city: ")
    if user_prompt.lower() in ['exit', 'quit']:
        print("Exiting...")
        break

    # 5. Define messages for a *new* conversation
    # LangChain uses message objects instead of dicts.
    messages = [
        SystemMessage(content="You are a helpful assistant. You must use the provided 'get_weather' tool to answer the user's request."),
        HumanMessage(content=f"What is the weather in {user_prompt}?")
    ]
    
    print(f"\nUser: {messages[1].content}\n")

    try:
        # 6. First call to the model
        # The model will see the prompt and the available tools.
        # Because of 'tool_choice', it's forced to return a tool_call.
        response_message = model_with_tools.invoke(messages)
        
        # Add the model's response (which is a tool call request) to history
        messages.append(response_message)
        
        print("[Model] Requested a function call...")

        # 7. Check the 'tool_calls' attribute on the response
        if not response_message.tool_calls:
            print("[Model] Did not request a tool call (this shouldn't happen with tool_choice).")
            print(f"{response_message.content}\n")
            continue

        # 8. Execute all requested tool calls
        for tool_call in response_message.tool_calls:
            print(f"  Function: {tool_call['name']}")
            print(f"  Arguments: {tool_call['args']}\n")
            
            # Find the actual function from our 'tools' list and run it
            if tool_call['name'] == "get_weather":
                function_response = get_weather.invoke(tool_call['args'])
                
                print(f"[Python] Sending function result back to model...\n")

                # 9. Send the function's result back to the model
                # We use a 'ToolMessage' that includes the output
                # and references the original 'tool_call_id'.
                messages.append(
                    ToolMessage(
                        content=function_response,
                        tool_call_id=tool_call['id']
                    )
                )

        # 10. Get the final, natural-language response
        # We call the model *again* with the full history,
        # which now includes the Human prompt, the AI's tool request,
        # and the tool's response.
        final_response = model_with_tools.invoke(messages)
        
        print(f"[Model] {final_response.content}\n")

    except Exception as e:
        print(f"[Error] An error occurred: {e}\n")



User: What is the weather in Pune?

[Model] Requested a function call...
  Function: get_weather
  Arguments: {'city': 'Pune', 'unit': 'C'}

[Python] Sending function result back to model...

[Model]  It is currently 23 degrees Celsius and sunny in Pune. Enjoy your day!

Exiting...


## 4- LangChain for Tool Calling
So far, you built a simple tool-calling pipeline manually. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward.

In this section, you will use the **ReAct** Agent (Reasoning + Acting). It alternates between reasoning steps and tool use, producing clearer and more reliable results. We will explore reasoning-focused models in more depth next week.

The following links might be helpful:
- https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html
- https://python.langchain.com/docs/integrations/tools/
- https://python.langchain.com/docs/integrations/chat/ollama/
- https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.llms.LLM.html

In [ ]:
# ---------------------------------------------------------
# Step 2: Initialize the LangChain Agent
# ---------------------------------------------------------
# Goal:
#   
#
# Steps:
#   1. Import the required classes:
#        - ChatOllama (for local model access)
#        - initialize_agent, Tool, AgentType
#   2. Create an LLM instance (e.g., model="gemma3:1b", temperature=0).
#   3. Add your tool(s) to a list
#   4. Initialize the agent using initialize_agent
#   5. Test the agent with a natural question (e.g., "Do I need an umbrella in Seattle today?").
#
# Expected:
#   The model should reason through the question, call your tool,
#   and produce a final answer in plain language.
# ---------------------------------------------------------

from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOllama(model="mistral:7b", temperature=0)

response = agent.run("What's the price for Adobe?")




User: What is the weather in Dubai?

[Model] Requested a function call...
  Function: get_weather
  Arguments: {'city': 'Dubai'}

[Python] Sending function result back to model...

[Model]  It is currently 23 degrees Celsius and sunny in Dubai. Enjoy your day!


User: What is the weather in bye?

[Model] Requested a function call...
  Function: get_weather
  Arguments: {'city': 'bye', 'unit': 'C'}

[Python] Sending function result back to model...

[Model]  It seems there might be a mistake. Bye is not a recognized city. Could you please provide the correct location?

Exiting...


### What just happened?

The console log displays the **Thought → Action → Observation → …** loop until the agent produces its final answer. Because `verbose=True`, LangChain prints each intermediate reasoning step.

If you want to add more tools, simply append them to the tools list. LangChain will handle argument validation, schema generation, and tool-calling logic automatically.


## 5- Perplexity‑Style Web Search
Agents become much more powerful when they can look up real information on the web instead of relying only on their internal knowledge.

In this section, you will combine everything you have learned to build a simple Ask-the-Web Agent. You will integrate a web search tool (DuckDuckGo) and make it available to the agent using the same tool-calling approach as before.

This will let the model retrieve fresh results, reason over them, and generate an informed answer—similar to how Perplexity works.

You may find some examples from the following links:
- https://pypi.org/project/duckduckgo-search/

In [10]:
# ---------------------------------------------------------
# Step 1: Add a web search tool
# ---------------------------------------------------------
# Goal:
#   Create a tool that lets the agent search the web and return results.
#
# Steps:
#   1. Use DuckDuckGo for quick, open web searches.
#   2. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DDGS to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   3. Wrap it with the @tool decorator to make it available to LangChain.


from ddgs import DDGS
from langchain_core.tools import tool

@tool
def search_web(query: str) -> str:
    """Search the web using DuckDuckGo and return top results."""
    results = DDGS().text(query, max_results=3)
    formatted_results = []
    for result in results:
        title = result.get('title', 'No Title')
        url = result.get('href', 'No URL')
        formatted_results.append(f"{title}: {url}")
    print(f"Search result count: {len(results)}")        
    return "\n".join(formatted_results) 

#results = search_web("Python programming")

tools.append(search_web)


llm = ChatOllama(model="mistral:7b", temperature=0)
model_with_tools = llm.bind_tools(tools=tools)

messages = [
        SystemMessage(content="You are a helpful assistant. Use the correct tool to answer the user's request."),
        HumanMessage(content=f"What are the 3 top news stories today?")
    ]
    
while True:
    user_prompt = input("Enter Web Search Text: ")
    if user_prompt.lower() in ['exit', 'quit']:
        print("Exiting...")
        break

    # 5. Define messages for a *new* conversation
    # LangChain uses message objects instead of dicts.
    messages = [
        SystemMessage(content="You are a helpful assistant. Use the correct tool to answer the user's request."),
        HumanMessage(content=f"Search the web for: {user_prompt}")
    ]
    
    print(f"\nUser: {messages[1].content}\n")

    try:
        # 6. First call to the model
        # The model will see the prompt and the available tools.
        # Because of 'tool_choice', it's forced to return a tool_call.
        response_message = model_with_tools.invoke(messages)
        
        # Add the model's response (which is a tool call request) to history
        messages.append(response_message)
        
        print("[Model] Requested a function call...")

        # 7. Check the 'tool_calls' attribute on the response
        if not response_message.tool_calls:
            print("[Model] Did not request a tool call (this shouldn't happen with tool_choice).")
            print(f"{response_message.content}\n")
            continue

        # 8. Execute all requested tool calls
        for tool_call in response_message.tool_calls:
            print(f"  Function: {tool_call['name']}")
            print(f"  Arguments: {tool_call['args']}\n")
            
            # Find the actual function from our 'tools' list and run it
            if tool_call['name'] == "get_weather":
                function_response = get_weather.invoke(tool_call['args'])
            if tool_call['name'] == "web_search":
                function_response = search_web.invoke(tool_call['args'])                
                print(f"[Python] Sending function result back to model...\n")

                # 9. Send the function's result back to the model
                # We use a 'ToolMessage' that includes the output
                # and references the original 'tool_call_id'.
                messages.append(
                    ToolMessage(
                        content=function_response,
                        tool_call_id=tool_call['id']
                    )
                )

        # 10. Get the final, natural-language response
        # We call the model *again* with the full history,
        # which now includes the Human prompt, the AI's tool request,
        # and the tool's response.
        final_response = model_with_tools.invoke(messages)
        
        print(f"[Model] {final_response.content}\n")

    except Exception as e:
        print(f"[Error] An error occurred: {e}\n")




User: Search the web for: top 3 news today

[Model] Requested a function call...
  Function: search_web
  Arguments: {'query': 'top 3 news today'}

[Model]   Here are the top 3 news articles I found based on your query:

1. Title: Biden signs $1.9 trillion COVID-19 relief bill into law (CNN)
   Link: https://www.cnn.com/2021/03/11/politics/biden-covid-relief-bill-signing/index.html

2. Title: Texas power grid operator says it could take weeks to restore electricity (Reuters)
   Link: https://www.reuters.com/world/us/texas-power-grid-operator-says-it-could-take-weeks-restore-electricity-2021-02-17/

3. Title: Facebook to ban ads promoting weapons sales (BBC News)
   Link: https://www.bbc.com/news/technology-56248903

These articles cover a range of topics including politics, natural disasters, and technology. The first article discusses President Biden signing the $1.9 trillion COVID-19 relief bill into law, the second article talks about the ongoing power outages in Texas, and the thi

In [ ]:

# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Goal:
#   Connect your `web_search` tool to a language model
#   so the agent can search and reason over real data.
#
# Steps:
#   1. Import `initialize_agent` and `AgentType`.
#   2. Create an LLM (e.g., ChatOllama).
#   3. Add your `web_search` tool to the tools list.
#   4. Initialize the agent using: initialize_agent
#   5. Keep `verbose=True` to observe reasoning steps.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI

"""
YOUR CODE HERE (~5 lines of code)
"""



Let’s see the agent's output in action with a real example.


In [ ]:

# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent
# ---------------------------------------------------------
# Goal:
#   Verify that the agent can search the web and return
#   a summarized answer based on real results.
#
# Steps:
#   1. Ask a natural question that requires live information,
#      for example: "What are the current events in San Francisco this week?"
#   2. Call agent.
#
# Expected:
#   The agent should call `web_search`, retrieve results,
#   and generate a short summary response.
# ---------------------------------------------------------

"""
YOUR CODE HERE (~2-5 lines of code)
"""




## 6- A minimal UI
This project includes a simple **React** front end that sends the user’s question to a FastAPI back end and streams the agent’s response in real time. To run the UI:

1- Open a terminal and start the Ollama server: `ollama serve`.

2- In a second terminal, navigate to the frontend folder and install dependencies:`npm install`.

3- In the same terminal, start the FastAPI back‑end: `uvicorn app:app --reload --port 8000`

4- Open a third terminal, stay in the frontend folder, and start the React dev server: `npm run dev`

5- Visit `http://localhost:5173/` in your browser.



## 🎉 Congratulations!

* You have built a **web‑enabled agent**: tool calling → JSON schema → LangChain ReAct → web search → simple UI.
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.